In [10]:
#importing dependencies

#importing Opencv for preprocessing
import cv2

#importing numpy for doing mathematical operation
import numpy as np

#importing matplotlib for seeing input and output image
import matplotlib.pyplot as plt

#importing scipy for calculating the similarity between images
from scipy import spatial

#importing python library for doing face detection as well as face recognition
import face_recognition

In [41]:
#required function for doing facial recignition and face alignment


######   FUCNTION 1   #######

#function to input in image, do it's alignment then return 128D embedding which contain important features about image which determines it's identity
def get_embedding(filename, i):
    
    image = face_recognition.load_image_file(filename)
    face_embedding = face_recognition.face_encodings(image)
    
    if len(face_embedding) != 0:
        return face_embedding[0]
    else:
        print('Sorry!, No face found in image', i)



######   FUCNTION 2   #######        

#function for comparing images in database and predicting top n similar images from database
def who_it_is(path_of_image, database, model, i):
    
    #storing similarity distance in dictionary 'similarity_distance' while storing name as well as distance in list 'list_of_similarity' for later sorting on basis of distance
    similarity_distance, list_of_similarity = {}, []
    
    #getting 128D encoding of input image from our trained model which includes important features, identity as well as important information about input image
    encoding = extract_face_and_get_embedding(path_of_image, 160, model, i)
    
    #initializing min_dist with some random larger number
    min_dist = 10
    
    #if no face is found in image than we are returning flag 1 and None which will tell us that in later function that there is no face in input image
    if encoding is None:
        list_of_similarity.append((1, None))
        return None, list_of_similarity
    
    #if face is found then we are doing furthur computation
    else:
        
        #searching throughout database and comparing with each image in database
        for (name, db_enc) in database.items():
            
            #if there is image in database where our model is not able to detect any face we are ignoring that image
            if db_enc is not None:

                #calculating similarity distance between images
                dist = spatial.distance.cosine(db_enc, encoding)
                
                #storing distance for later use
                similarity_distance[name] = dist

                list_of_similarity.append((dist, name))

                #storing image of minimum distacne in the database
                if dist < min_dist:

                    min_dist = dist
                    identity = name

        #returning similarity distance(a dictonary)
        return similarity_distance, list_of_similarity
    


######   FUCNTION 3   #######    
    
    
#function to show top n similar image as well as input image and their similarity distances with their name in database
def output(image_path, similarity_distance, list_of_similarity, number_of_images_to_show):
    
    #showing input image
    plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
    plt.title('Input Image')
    plt.show()
    
    if len(list_of_similarity) != 0:
        #sorting similarity in ascending order to take out top n minimum similarity distances
        list_of_similarity.sort()
        print('\n\n')
        #showing best n similar images found from databse
        print('TOP '+str(number_of_images_to_show)+' BEST IMAGES FOUND FROM DATABASE ARE :-----')
        for i in range(number_of_images_to_show):
            print('\n\n')
            print('Similar Image '+str(i+1)+' found is: '+str(list_of_similarity[i][1])+'.jpg with similarity '+str(round(list_of_similarity[i][0], 4)))
            plt.imshow(cv2.cvtColor(cv2.imread('Database/'+str(list_of_similarity[i][1])+'.jpg'), cv2.COLOR_BGR2RGB))
            plt.show()
            print('\n\n')
        
    else:
        print('Sorry!, No face found in image')

In [1]:
%%time
#creating embeddings of all the images in database and storing in variable 'database' to save at the time of 
#outputing the images otherwise it will take around 15 min for giving output of each image beacause it has to calculate 
#embedding each time of whole database.

database = {}
for i in range(169):
    database[i+1] = get_embedding('Database/'+str(i+1)+'.jpg', i+1)

In [2]:
#predicting top n images found similar to input image
for i in range(1, 122):
    
    image_path, image_name = 'Test Images/'+str(i)+'.jpg', i
    output(image_path, *who_it_is(image_path, database, image_name), 1)
    print('\n\n')